In [1]:
import pandas as pd
import sqlite3 

In [2]:
!pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#Joining dataset and reading csv to pandas dataframe:
df = pd.read_csv('cyclistic_data.csv')
df2 = pd.read_csv('cyclistic_c_data.csv') 
combined_df = pd.concat([df, df2], ignore_index=True)

In [ ]:
cnn = sqlite3.connect('simsimb.db')

In [ ]:
combined_df.to_sql('cyclisticdata', cnn, if_exists='replace')

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///simsimb.db

In [ ]:
%%sql
SELECT *
FROM 
cyclisticdata
limit 10;

In [ ]:
%%sql
UPDATE cyclisticdata
SET month = CASE month
                  WHEN '1' THEN 'January'
                  WHEN '2' THEN 'February'
                  WHEN '3' THEN 'March'
                  WHEN '4' THEN 'April'
                  WHEN '5' THEN 'May'
                  WHEN '6' THEN 'June'
                  WHEN '7' THEN 'July'
                  WHEN '8' THEN 'August'
                  WHEN '9' THEN 'September'
                  WHEN '10' THEN 'October'
                  WHEN '11' THEN 'November'
                  WHEN '12' THEN 'December'
                  ELSE month  
                END
WHERE month IN ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12');

In [ ]:
%%sql
UPDATE cyclisticdata
SET dayofweek = CASE dayofweek
                  WHEN '0' THEN 'Monday'
                  WHEN '1' THEN 'Tuesday'
                  WHEN '2' THEN 'Wednesday'
                  WHEN '3' THEN 'Thursday'
                  WHEN '4' THEN 'Friday'
                  WHEN '5' THEN 'Saturday'
                  WHEN '6' THEN 'Sunday'
                  ELSE dayofweek  
                END
WHERE dayofweek IN ('0', '1', '2', '3', '4', '5', '6');

In [ ]:
%%sql 
SELECT COUNT(rideid) totaltrips,
SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) totalmember, 
SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) totalcasual
FROM cyclisticdata;

In [ ]:
%%sql 
SELECT COUNT(CASE WHEN rideid IS NULL THEN 1 END) AS null_count
FROM cyclisticdata;

In [ ]:
%%sql
SELECT totaltrips, totalmember, totalcasual,
       ROUND(((totalcasual * 100) / totaltrips), 2) AS casualpercentage, 
       ROUND(CAST((totalmember * 100) / totaltrips AS FLOAT), 2) AS memberpercentage  
FROM (
    SELECT COUNT(rideid) AS totaltrips,
           SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS totalmember,
           SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS totalcasual
    FROM cyclisticdata
)

--of the 1568271 trips, 67percent were members and 32 were casual

In [ ]:
%%sql
SELECT
    (SELECT ROUND(AVG(durationmins), 2)
     FROM cyclisticdata) AS overallavg,
    membercasual,
    avgrideduration
FROM (
    SELECT membercasual, ROUND(AVG(durationmins), 2) AS avgrideduration
    FROM cyclisticdata
    WHERE membercasual IN ('member', 'casual')
    GROUP BY membercasual
) AS subquery;

In [ ]:
%%sql
SELECT  membercasual, ROUND(MAX(durationmins),0) AS maxduration
FROM cyclisticdata
GROUP BY membercasual
ORDER BY maxduration DESC;

In [ ]:
%%sql
SELECT membercasual, MIN(durationmins) AS minimduration
FROM cyclisticdata
GROUP BY membercasual
ORDER BY minimduration;

In [ ]:
%%sql
SELECT membercasual, dayofweek AS busiestweekday
FROM (
    SELECT DISTINCT membercasual, dayofweek,
           ROW_NUMBER() OVER (PARTITION BY membercasual ORDER BY COUNT(dayofweek) DESC) AS dowpopularity 
    FROM cyclisticdata
    GROUP BY membercasual, dayofweek
) AS subquery  -- Added alias for subquery
WHERE dowpopularity = 1
ORDER BY membercasual DESC
LIMIT 2;

In [ ]:
%%sql
    SELECT dayofweek, COUNT(rideid) AS totaltrips,
           SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS totalmember,
           SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS totalcasual
    FROM cyclisticdata
    GROUP BY dayofweek
    ORDER BY totaltrips DESC;

In [ ]:
%%sql
SELECT startstationname, 
       SUM(CASE WHEN rideid = rideid AND startstationname = startstationname THEN 1 ELSE 0 END) AS startstationcount,
       --SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberstationcount,  
       SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualstationcount  
FROM cyclisticdata
GROUP BY startstationname
ORDER BY startstationcount DESC
LIMIT 20;

In [ ]:
%%sql
SELECT startstationname, 
       SUM(CASE WHEN rideid = rideid AND startstationname = startstationname THEN 1 ELSE 0 END) AS startstationcount,
       --SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberstationcount,  
       SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualstationcount  
FROM cyclisticdata
GROUP BY startstationname
ORDER BY casualstationcount DESC
LIMIT 10;

In [ ]:
%%sql
SELECT startstationname, 
       SUM(CASE WHEN rideid = rideid AND startstationname = startstationname THEN 1 ELSE 0 END) AS startstationcount,
       SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberstationcount 
       --SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualstationcount  
FROM cyclisticdata
GROUP BY startstationname
ORDER BY memberstationcount DESC
LIMIT 10;

In [ ]:
%%sql
SELECT endstationname, 
       SUM(CASE WHEN rideid = rideid AND endstationname = endstationname THEN 1 ELSE 0 END) AS endstationcount,
       --SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberendstationcount,  
       SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualendstationcount  
FROM cyclisticdata
GROUP BY endstationname
ORDER BY endstationcount DESC
LIMIT 20;

In [ ]:
%%sql
SELECT startstationname, 
       SUM(CASE WHEN rideid = rideid AND endstationname = endstationname THEN 1 ELSE 0 END) AS endstationcount,
       SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberendstationcount,  
       --SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualendstationcount  
FROM cyclisticdata
GROUP BY endstationname
ORDER BY memberendstationcount DESC
LIMIT 10;

In [ ]:
%%sql
SELECT endstationname, 
       SUM(CASE WHEN rideid = rideid AND endstationname = endstationname THEN 1 ELSE 0 END) AS endstationcount,
       --SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberendstationcount,  
       SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualendstationcount  
FROM cyclisticdata
GROUP BY endstationname
ORDER BY casualendstationcount DESC
LIMIT 10;

In [ ]:
combined.to_csv('cyclistic1_data.csv')